In [ ]:
# jupyter_vim

In [1]:
# Run Script below
! export PYTHONPATH=../risingwave:$PYTHONPATH && python ../risingwave/src/main.py --topic=book --bootstrap-servers=kafka-broker:9092 --schema='{"line": "INTEGER","text": "VARCHAR"}' --source='kafka'

In [2]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(host="risingwave", port=4566, user="root", dbname="dev")
conn.autocommit = True

In [3]:
with conn.cursor() as cur:
    cur.execute("SELECT kafka_timestamp, line, text FROM kafka_view LIMIT 10;")
    values = cur.fetchall()

In [4]:
pd.DataFrame(values, columns=["timestamp", "line", "text"])

,timestamp,line,text
0,2025-03-08 05:39:39.926000+00:00,1,The Project Gutenberg eBook of The Brothers Ka...
1,2025-03-08 05:39:39.926000+00:00,2,
2,2025-03-08 05:39:39.926000+00:00,3,This ebook is for the use of anyone anywhere i...
3,2025-03-08 05:39:39.926000+00:00,4,most other parts of the world at no cost and w...
4,2025-03-08 05:39:39.926000+00:00,5,"whatsoever. You may copy it, give it away or r..."
5,2025-03-08 05:39:39.926000+00:00,6,of the Project Gutenberg License included with...
6,2025-03-08 05:39:39.926000+00:00,7,at www.gutenberg.org. If you are not located i...
7,2025-03-08 05:39:39.926000+00:00,8,you will have to check the laws of the country...
8,2025-03-08 05:39:39.926000+00:00,9,before using this eBook.
9,2025-03-08 05:39:39.926000+00:00,10,


In [5]:
import duckdb

con = duckdb.connect()
con.sql(
    f"""
    INSTALL postgres;
    LOAD postgres;  
    ATTACH 'host=risingwave port=4566 dbname=dev user=root' AS db (TYPE postgres);  
"""
)
con.sql("SHOW DATABASES;")
con.sql("SHOW ALL TABLES;")

Error: Failed to execute query "BEGIN TRANSACTION ISOLATION LEVEL REPEATABLE READ;

SELECT oid, nspname
FROM pg_namespace

ORDER BY oid;

SELECT pg_namespace.oid AS namespace_id, relname, relpages, attname,
    pg_type.typname type_name, atttypmod type_modifier, pg_attribute.attndims ndim,
    attnum, pg_attribute.attnotnull AS notnull, NULL constraint_id,
    NULL constraint_type, NULL constraint_key
FROM pg_class
JOIN pg_namespace ON relnamespace = pg_namespace.oid
JOIN pg_attribute ON pg_class.oid=pg_attribute.attrelid
JOIN pg_type ON atttypid=pg_type.oid
WHERE attnum > 0 AND relkind IN ('r', 'v', 'm', 'f', 'p') 
UNION ALL
SELECT pg_namespace.oid AS namespace_id, relname, NULL relpages, NULL attname, NULL type_name,
    NULL type_modifier, NULL ndim, NULL attnum, NULL AS notnull,
    pg_constraint.oid AS constraint_id, contype AS constraint_type,
    conkey AS constraint_key
FROM pg_class
JOIN pg_namespace ON relnamespace = pg_namespace.oid
JOIN pg_constraint ON (pg_class.oid=pg_con